Version: 02.14.2023

# Laboratório 2.1: Aplicação do ML a um problema de PLN

Neste laboratório, você usará o modelo de machine learning (ML) integrado ao Amazon SageMaker, __LinearLearner__, para prever o campo __isPositive__ do conjunto de dados da avaliação.

## Apresentação do cenário de negócios
Você trabalha para uma loja de varejo on-line que quer melhorar o engajamento dos clientes que publicaram avaliações negativas. A empresa quer detectar avaliações negativas e atribuir a elas a um agente de atendimento ao cliente para tratar do assunto.

Sua tarefa é usar o ML para detectar avaliações negativas e resolver parte desse problema. Você recebeu acesso a um conjunto de dados que contém avaliações, que foram classificadas como positivas ou negativas. Use esse conjunto de dados para treinar um modelo de ML e prever o sentimento das próximas avaliações.

## Sobre esse conjunto de dados
O arquivo [AMAZON-REVIEW-DATA-CLASSIFICATION.csv](https://github.com/aws-samples/aws-machine-learning-university-accelerated-nlp/tree/master/data/examples) contém avaliações reais de produtos, e elas contêm dados de texto e dados numéricos. Cada avaliação é classificada como _positive (1)_ ou _negative (0)_.

O conjunto de dados contém os seguintes recursos:
* __reviewText:__ texto da avaliação
* __summary:__ resumo da avaliação
* __verified:__ se a compra foi verificada (Verdadeiro ou Falso)
* __time:__ descrição de data/hora UNIX da avaliação
* __log_votes:__ registro de votos ajustado por logaritmo (1+votos)
* __isPositive:__ se a avaliação é positiva ou negativa (1 ou 0)

O conjunto de dados deste laboratório está sendo fornecido a você com permissão da Amazon e está sujeito aos termos de Licença e acesso da Amazon (disponíveis em https://www.amazon.com/gp/help/customer/display.html?nodeId=201909000). É expressamente proibido copiar, modificar, vender, exportar ou usar este conjunto de dados com outro propósito que não seja para fins de conclusão deste laboratório.

## Etapas do laboratório

Para concluir o laboratório, siga estas etapas:

1. [Leitura do conjunto de dados](#1.-Reading-the-dataset)
2. [Realização da análise exploratória de dados](#2.-Performing-exploratory-data-analysis)
3. [Processamento de texto: remoção de palavras irrelevantes e stemming](#3.-Text-processing:-Removing-stopwords-and-stemming)
4. [Divisão dos dados para treinamento, validação e teste](#4.-Splitting-training,-validation,-and-test-data)
5. [Processamento de dados com pipelines e um ColumnTransformer](#5.-Processing-data-with-pipelines-and-a-ColumnTransformer)
6. [Treinamento de um classificador com um algoritmo integrado ao SageMaker](#6.-Training-a-classifier-with-a-built-in-SageMaker-algorithm)
7. [Avaliação do modelo](#7.-Evaluating-the-model)
8. [Implementação do modelo em um endpoint](#8.-Deploying-the-model-to-an-endpoint)
9. [Teste do endpoint](#9.-Testing-the-endpoint)
10. [Remoção de artefatos do modelo](#10.-Cleaning-up-model-artifacts)
    
## Envio do trabalho

1. No console do laboratório, escolha **Submit** (Enviar) para registrar seu progresso e, quando solicitado, escolha **Yes** (Sim).

1. Se os resultados não forem exibidos após alguns minutos, volte ao topo destas instruções e escolha **Grades** (Notas).

     **Dica**: você pode enviar seu trabalho várias vezes. Depois de alterar o trabalho, escolha **Submit** (Enviar) novamente. Seu último envio é o que será gravado para o laboratório.

1. Para ver o feedback detalhado sobre seu trabalho, escolha **Details** (Detalhes) e depois **View Submission Report** (Ver relatório do envio).  

Primeiro, instale/atualize o pip, o sagemaker e a scikit-learn.

[scikit-learn] (https://scikit-learn.org/stable/) é uma biblioteca de machine learning de código aberto. Ela oferece várias ferramentas para ajustar modelos, pré-processar dados, selecionar e avaliar modelos e muitos outros utilitários.

In [ ]:
#Upgrade dependencies
!pip install --upgrade pip
!pip install --upgrade scikit-learn
!pip install --upgrade sagemaker
!pip install --upgrade botocore
!pip install --upgrade awscli

## 1. Leitura do conjunto de dados
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Use a biblioteca __pandas__ para ler o conjunto de dados. [Pandas] (https://pandas.pydata.org/pandas-docs/stable/index.html) é uma biblioteca Python popular usada para analisar dados. Ela oferece recursos de manipulação de dados, limpeza e controle de dados, além de visualizações.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/AMAZON-REVIEW-DATA-CLASSIFICATION.csv')

print('The shape of the dataset is:', df.shape)

Observe as primeiras cinco linhas do conjunto de dados.

In [ ]:
df.head(5)

É possível alterar as opções na atividade para exibir mais dados de texto.

In [ ]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)
df.head()

Você pode analisar entradas específicas se necessário.

In [ ]:
print(df.loc[[580]])

É importante saber com quais tipos de dados você está lidando. Você pode usar  `dtypes` no dataframe para exibir os tipos.

In [ ]:
df.dtypes

## 2. Realização da análise exploratória de dados
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Agora você verá a distribuição de destinos do conjunto de dados.

In [ ]:
df['isPositive'].value_counts()

O problema de negócios é identificar as avaliações negativas (_0_). Porém, o ajuste do modelo para o linear learner encontra valores positivos (_1_) por padrão. Você pode trocar os valores negativos (_0_) e os valores positivos (_1_) para fazer esse processo funcionar melhor. Assim, será possível ajustar o modelo com mais facilidade.

In [ ]:
df = df.replace({0:1, 1:0})
df['isPositive'].value_counts()

Confira o número de valores ausentes:

In [ ]:
df.isna().sum()

Os campos de texto têm valores ausentes. Geralmente, você decide o que fazer com esses valores. É possível remover os dados ou preenchê-los com um texto-padrão. 

## 3. Processamento de texto: remoção de palavras irrelevantes e stemming
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Nesta tarefa, você removerá algumas das palavras irrelevantes e fará o stemming nos dados de texto. Você normalizará os dados para reduzir a quantidade de informações diferentes com as quais é preciso lidar.

[nltk](https://www.nltk.org/) é uma plataforma popular para trabalhar com dados de linguagem humana. Ela oferece interfaces e funções para processar texto para classificação, tokenização, stemming, marcação, análise e raciocínio semântico. 

Depois de importá-la, você só pode baixar a funcionalidade necessária. Neste exemplo, você usará:

- **punkt** é um tokenizador de frases.
- **stopwords** apresenta uma lista de palavras irrelevantes que você pode usar.

In [ ]:
# Install the library and functions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

Na seção a seguir, você criará os processos para remover as palavras irrelevantes e limpar o texto. A biblioteca Natural Language Toolkit (NLTK) oferece uma lista de palavras irrelevantes comuns. Você usará a lista, mas primeiro removerá algumas palavras dela. As palavras irrelevantes mantidas no texto são úteis para identificar o sentimento.

In [ ]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Get a list of stopwords from the NLTK library
stop = stopwords.words('english')

# These words are important for your problem. You don't want to remove them.
excluding = ['against', 'not', 'don', 'don\'t','ain', 'are', 'aren\'t', 'could', 'couldn\'t',
             'did', 'didn\'t', 'does', 'doesn\'t', 'had', 'hadn\'t', 'has', 'hasn\'t', 
             'have', 'haven\'t', 'is', 'isn\'t', 'might', 'mightn\'t', 'must', 'mustn\'t',
             'need', 'needn\'t','should', 'shouldn\'t', 'was', 'wasn\'t', 'were', 
             'weren\'t', 'won\'t', 'would', 'wouldn\'t']

# New stopword list
stopwords = [word for word in stop if word not in excluding]




O stemizador snowball stemiza as palavras. Por exemplo, o verbo “caminhando” será transformado em “caminh”.

In [ ]:
snow = SnowballStemmer('english')

É preciso realizar outras tarefas de normalização nos dados. A seguinte função:

- Substitui todos os valores ausentes por uma string vazia
- Converte o texto em letras minúsculas
- Remove espaços em branco à esquerda ou à direita
- Remove espaços e recuos extras
- Remove marcações HTML

No loop  `for`, todas as palavras que __NOT__ são numéricas, têm mais de dois caracteres e não estão na lista de palavras irrelevantes são mantidas e retornadas.

In [ ]:
def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ''
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        
        for w in word_tokenize(sent):
            # Applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stopwords
            if(not w.isnumeric()) and (len(w)>2) and (w not in stopwords):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) # Final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

## 4. Divisão dos dados para treinamento, validação e teste
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Nesta etapa, você usará a função sklearn [__train_test_split()__](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) a fim de dividir o conjunto de dados para treinamento (80%), validação (10%) e teste (10%).

Os dados de treinamento serão usados para treinar o modelo, que depois será testado com os dados de teste. O conjunto de validação é usado depois que o modelo foi treinado para apresentar métricas sobre a performance do modelo em dados reais. 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df[['reviewText', 'summary', 'time', 'log_votes']],
                                                  df['isPositive'],
                                                  test_size=0.20,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

X_val, X_test, y_val, y_test = train_test_split(X_val,
                                                y_val,
                                                test_size=0.5,
                                                shuffle=True,
                                                random_state=324)

Agora, com o conjunto de dados dividido, você pode executar a função  `process_text` definida acima em cada um dos recursos de texto nos conjuntos de treinamento, teste e validação.

In [ ]:
print('Processing the reviewText fields')
X_train['reviewText'] = process_text(X_train['reviewText'].tolist())
X_val['reviewText'] = process_text(X_val['reviewText'].tolist())
X_test['reviewText'] = process_text(X_test['reviewText'].tolist())

print('Processing the summary fields')
X_train['summary'] = process_text(X_train['summary'].tolist())
X_val['summary'] = process_text(X_val['summary'].tolist())
X_test['summary'] = process_text(X_test['summary'].tolist())

## 5. Processamento de dados com pipelines e um ColumnTransformer
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Geralmente, você precisa realizar muitas tarefas nos dados antes de usá-los para treinar um modelo. Essas etapas também devem ser feitas em todos os dados usados para inferência após a implementação do modelo. Uma boa forma de organizar as etapas é definir um _pipeline_. Um pipeline é uma coleção de tarefas de processamento que serão realizadas nos dados. É possível criar pipelines diferentes para processar campos diferentes. Como você está trabalhando com dados numéricos e de texto, defina os seguintes pipelines:

   * Para o pipeline de recursos numéricos, o __numerical_processor__ usa um MinMaxScaler. (Não é necessário dimensionar os recursos ao usar árvores de decisão, mas é importante fazer isso para saber como usar mais transformações de dados.) Caso você queira realizar diferentes tipos de processamento em diferentes recursos numéricos, crie pipelines diferentes, como os que são mostrados para os dois recursos de texto.
   * Para o pipeline de recursos de texto, o __text_processor__ usa  `CountVectorizer()` nos campos de texto.
   
As preparações seletivas dos recursos do conjunto de dados são reunidas em um ColumnTransformer coletivo, que será usado em um pipeline com um estimador. Esse processo garante que as transformações sejam realizadas automaticamente nos dados brutos quando você ajusta o modelo ou faz previsões. (Por exemplo, ao avaliar o modelo em um conjunto de dados de validação por meio da validação cruzada ou ao fazer previsões em um conjunto de dados de teste no futuro.)

In [ ]:
# Grab model features/inputs and target/output
numerical_features = ['time',
                      'log_votes']

text_features = ['summary',
                 'reviewText']

model_features = numerical_features + text_features
model_target = 'isPositive'

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

### COLUMN_TRANSFORMER ###
##########################

# Preprocess the numerical features
numerical_processor = Pipeline([
    ('num_imputer', SimpleImputer(strategy='mean')),
    ('num_scaler', MinMaxScaler()) 
                                ])
# Preprocess 1st text feature
text_processor_0 = Pipeline([
    ('text_vect_0', CountVectorizer(binary=True, max_features=50))
                                ])

# Preprocess 2nd text feature (larger vocabulary)
text_precessor_1 = Pipeline([
    ('text_vect_1', CountVectorizer(binary=True, max_features=150))
                                ])

# Combine all data preprocessors from above (add more, if you choose to define more!)
# For each processor/step specify: a name, the actual process, and finally the features to be processed
data_preprocessor = ColumnTransformer([
    ('numerical_pre', numerical_processor, numerical_features),
    ('text_pre_0', text_processor_0, text_features[0]),
    ('text_pre_1', text_precessor_1, text_features[1])
                                    ]) 

### DATA PREPROCESSING ###
##########################

print('Datasets shapes before processing: ', X_train.shape, X_val.shape, X_test.shape)

X_train = data_preprocessor.fit_transform(X_train).toarray()
X_val = data_preprocessor.transform(X_val).toarray()
X_test = data_preprocessor.transform(X_test).toarray()

print('Datasets shapes after processing: ', X_train.shape, X_val.shape, X_test.shape)

Observe que o número de recursos nos conjuntos de dados passou de 4 para 202.

In [ ]:
print(X_train[0])

## 6. Treinamento de um classificador com um algoritmo integrado ao SageMaker
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Nesta etapa, você chamará o algoritmo  `LinearLearner()` do SageMaker com as seguintes opções:
* __Permissions -__  `role` está configurado como a role do AWS Identity and Access Management (IAM) do ambiente atual.
* __Compute power -__ Você usará os parâmetros  `train_instance_count` e  `train_instance_type`. Este exemplo usa um recurso  `ml.m4.xlarge` para o treinamento. É possível alterar o tipo de instância dependendo de suas necessidades. (Por exemplo, você pode usar GPUs para redes neurais.) 
* __Model type -__  `predictor_type` está configurado como __`binary_classifier`__, porque você está lidando com um problema de classificação binária. É possível usar __`multiclass_classifier`__ se há três ou mais classes envolvidas ou usar __`regressor`__ para um problema de regressão.


In [ ]:
import sagemaker

# Call the LinearLearner estimator object
linear_classifier = sagemaker.LinearLearner(role=sagemaker.get_execution_role(),
                                           instance_count=1,
                                           instance_type='ml.m4.xlarge',
                                           predictor_type='binary_classifier')

Para definir as partes de treinamento, validação e teste do estimador, use a função  `record_set()` do  `binary_estimator`. 

In [ ]:
train_records = linear_classifier.record_set(X_train.astype('float32'),
                                            y_train.values.astype('float32'),
                                            channel='train')
val_records = linear_classifier.record_set(X_val.astype('float32'),
                                          y_val.values.astype('float32'),
                                          channel='validation')
test_records = linear_classifier.record_set(X_test.astype('float32'),
                                           y_test.values.astype('float32'),
                                           channel='test')

A função  `fit()` aplica uma versão distribuída do algoritmo Stochastic Gradient Descent (SGD), e você envia os dados para ele. Os registros foram desabilitados com  `logs=False`. É possível remover esse parâmetro para ver mais detalhes sobre o processo. __Esse processo leva cerca de 3 a 4 minutos em uma instância ml.m4.xlarge.__

In [ ]:
linear_classifier.fit([train_records,
                       val_records,
                       test_records],
                       logs=False)

## 7. Avaliação do modelo
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Use a análise do SageMaker para ver métricas de performance (de sua escolha) do conjunto de testes. Esse processo não exige a implementação do modelo. 

O linear learner apresenta métricas que são calculadas durante o treinamento. É possível usar essas métricas ao ajustar o modelo. As métricas disponíveis para o conjunto de validação são:

- objective_loss - No caso de um problema de classificação binária, será o valor médio da perda logística para cada época
- binary_classification_accuracy - A precisão do modelo final no conjunto de dados, ou seja, quantas previsões o modelo acertou
- precision - Quantifica o número de previsões de classes positivas que são de fato positivas
- recall - Quantifica o número de previsões de classes positivas
- binary_f_beta - A média harmônica das métricas de precisão e recall

Neste exemplo, é importante observar quantas previsões estavam corretas. O uso da métrica **binary_classification_accuracy** é apropriado.

In [ ]:
sagemaker.analytics.TrainingJobAnalytics(linear_classifier._current_job_name, 
                                         metric_names = ['test:binary_classification_accuracy']
                                        ).dataframe()

O valor aproximado de 0,85 deve aparecer. O número pode ser diferente, mas deve estar em torno desse valor. Isso quer dizer que o modelo prevê, com precisão, a resposta correta em 85% do tempo. Dependendo do caso de negócios, talvez seja necessário ajustar os hiperparâmetros para melhorar o modelo ainda mais ou fazer engenharia de recursos.

## 8. Implementação do modelo em um endpoint
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Nesta última parte do exercício, você implementará o modelo em outra instância de sua escolha. É possível usar esse modelo em um ambiente de produção. Os endpoints implementados podem ser usados com outros serviços da AWS, como o AWS Lambda e o Amazon API Gateway. Para saber mais, veja a seguinte demonstração: [Chamar um endpoint de modelo do Amazon SageMaker usando o Amazon API Gateway e o AWS Lambda] (https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/).

Para implementar o modelo, execute a seguinte célula: É possível usar diferentes tipos de instância, como: _ml.t2.medium_, _ml.c4.xlarge_), entre outras. __Esse processo levará de 7 a 8 minutos.__

In [ ]:

linear_classifier_predictor = linear_classifier.deploy(initial_instance_count = 1,
                                                       instance_type = 'ml.c5.large'
                                                      )

## 9. Teste do endpoint
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Agora que o endpoint está implementado, você enviará os dados de teste para ele e verá previsões dos dados.

In [ ]:
import numpy as np

# Get test data in batch size of 25 and make predictions.
prediction_batches = [linear_classifier_predictor.predict(batch)
                      for batch in np.array_split(X_test.astype('float32'), 25)
                     ]

# Get a list of predictions
print([pred.label['score'].float32_tensor.values[0] for pred in prediction_batches[0]])

## 10. Remoção de artefatos do modelo
([Ir para o topo](#Lab-2.1:-Applying-ML-to-an-NLP-Problem))

Siga as etapas abaixo para excluir o endpoint depois que terminar de usá-lo. 

**Dica:** lembre-se de que, se você usar sua própria conta, serão registradas cobranças caso você não exclua o endpoint e outros recursos.

In [ ]:
linear_classifier_predictor.delete_endpoint()

# Parabéns!

Neste laboratório, você acompanhou um problema de PLN muito simples. Com um conjunto de dados classificado, você usou um tokenizador e um codificador simples para gerar os dados necessários para treinar um modelo de linear learner. Depois, você implementou o modelo e fez algumas previsões. Se você estivesse realizando esse processo de verdade, provavelmente precisaria obter os dados e identificá-los para o treinamento. Uma opção é usar um algoritmo pré-treinado ou um serviço gerenciado. Também provavelmente seria necessário ajustar os hiperparâmetros para melhorar o modelo ainda mais.

Você concluiu o laboratório e agora pode encerrá-lo seguindo as instruções do guia.

©2023 Amazon Web Services, Inc. ou suas afiliadas. Todos os direitos reservados. Este trabalho não pode ser reproduzido ou redistribuído, no todo ou em parte, sem a permissão prévia por escrito da Amazon Web Services, Inc. É proibido copiar, emprestar ou vender para fins comerciais. Todas as marcas comerciais pertencem a seus proprietários.